In [2]:
import json
import pandas as pd
from scipy.stats import entropy
from pandas import Series, DataFrame
import matplotlib.dates as mdates
import records
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.parser import parse

In [3]:
#年龄计算函数
def calculate_age(birthDate):
    days_in_year = 365.2425
    age = int((datetime.datetime.today() - birthDate).days / days_in_year)
    return age

In [7]:
#提取数据， 解析json
with records.Database(
            'mysql+pymysql://makex:P@ssw0rd@rm-wz945n00bvy1fbj16oo.mysql.rds.aliyuncs.com:3306/2019_data_analyse?charset=utf8') as db:
    db.query('use 2019_data_analyse;')

    regs = json.loads(db.query("select * from regs inner join games on regs.game_id = games.id ").export('json'))
    game_users = []
    for reg in regs:
        for user in json.loads(reg['users']):
            if user['teamRole'] == 'teacher':
                continue
            game_users.append({
                'reg_id' : reg['id'],
                'birthday' : user['birthday'],
                'game_type_id': reg['game_type_id'],
                'user_name':user['name'],
                'team_id':reg['team_id']
            })
    df = DataFrame.from_records(game_users)
    df['age'] = df['birthday'].apply(lambda a: calculate_age(parse(a)))
df.to_excel('勇者征途年龄.xlsx')
#     #分类画图
#     gdf = df.groupby(['game_type_id','age'])['reg_id'].count().reset_index()
#     print(gdf)
#     data = gdf[gdf['game_type_id'] == 5]
#     print(data)
#     sns.barplot(x=data['age'],y=data['reg_id'])
#     plt.show()